手写Vit

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/b212/anaconda3/envs/pytorch_gpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
def ImgToPatchNative(img, patch_size, weight):
    output = F.unfold(img, kernel_size=patch_size, stride=patch_size).transpose(1, 2)
    embedding = torch.matmul(output, weight)
    return embedding
    
def ImgToPatchConv(img, kernel, stride):
    conv_output = F.conv2d(img, kernel, stride=stride)
    bs, oc, h, w = conv_output.shape
    embedding = conv_output.reshape(bs, oc, h*w).transpose(1,2)
    return embedding
    
    
    
bs, ic, h, w = 2, 3, 64, 64
patch_size = 8
model_dim = 8
weight_h = ic*patch_size*patch_size
label = torch.randint(10, (bs,))
# print(label)

img = torch.rand(bs, ic, h, w)
weight = torch.rand(weight_h, model_dim, requires_grad=True)
print(img.shape)
print(weight.shape)
output_native = ImgToPatchNative(img, patch_size, weight)

#oc, ic, (size)
kernel = weight.transpose(0,1).reshape(-1, ic, patch_size, patch_size)
print(kernel.shape)
output_conv = ImgToPatchConv(img, kernel, stride=8)
# print(output_conv.shape)
print(torch.allclose(output_native, output_conv))
print(output_native.shape)
print(output_conv.shape)

tensor([9, 9])
torch.Size([2, 3, 64, 64])
torch.Size([192, 8])
torch.Size([8, 3, 8, 8])
True
torch.Size([2, 64, 8])
torch.Size([2, 64, 8])


In [23]:
#step2 prepared CLS token embedding
cls_token_embedding = torch.rand(bs, 1, model_dim, requires_grad = True)
token_embedding = torch.cat([cls_token_embedding, output_conv], dim = 1)
print(token_embedding.shape)

torch.Size([2, 65, 8])


In [24]:
max_num_token = 128
#step3 add position embedding
positon_embedding_table = torch.rand(max_num_token, model_dim, requires_grad = True)
seq_len = token_embedding.shape[1]
position_embedding = torch.tile(positon_embedding_table[:seq_len], [token_embedding.shape[0], 1, 1])
token_embedding += position_embedding

In [25]:
#step Transformer Layer
encode_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=8)
transformer_encoder = nn.TransformerEncoder(encode_layer, num_layers=6)
out = transformer_encoder(token_embedding)
print(out.shape)
cls_token_output = out[:,0,:]
linear_layer = nn.Linear(model_dim, num_class)
logits = linear_layer(cls_token_output)
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits,label)
print(loss)

torch.Size([2, 65, 8])


NameError: name 'num_class' is not defined